In [1]:
import pymongo
from pymongo import MongoClient
import dateutil.parser
import datetime
import pprint
import csv
import os
import pandas as pd
import json
from bson import BSON
from bson import json_util

In [2]:
client = MongoClient('localhost', 27017)
db=client["DM_project_database"]

In [3]:
#query

In [4]:
def ritardo_medio_weekday(filename,collection,time_start="00:00:00",time_end="23:59:59",highway=True):
    mycol=db[collection]
    time_start=datetime.datetime.strptime(time_start,"%H:%M:%S")
    time_end=datetime.datetime.strptime(time_end,"%H:%M:%S")
    if(highway==False):
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"path":"$_id",
                                        "day_of_week":"$traffic_info.day_of_week"},"avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    else:
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":"$traffic_info.day_of_week","avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        df.to_json("C:/Users/dasan/Desktop/query_new/{}.json".format(filename),orient='table')
        



In [5]:
def ritardo_medio_per_day(filename,collection,time_start="00:00:00",time_end="23:59:59",highway=True):
    mycol=db[collection]
    time_start=datetime.datetime.strptime(time_start,"%H:%M:%S")
    time_end=datetime.datetime.strptime(time_end,"%H:%M:%S")
    if(highway==False):
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"path":"$_id",
                                        "day":"$traffic_info.date"},"avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    else:
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":"$traffic_info.date","avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        df.to_json("C:/Users/dasan/Desktop/query_new/{}.json".format(filename),orient='table')

In [6]:
def ritardo_medio_per_hour(filename,collection,time_start,time_end,month=True):
    mycol=db[collection]
    time_start=datetime.datetime.strptime(time_start,"%H:%M:%S")
    time_end=datetime.datetime.strptime(time_end,"%H:%M:%S")
    
    if(month==False):
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"path":"$_id", "from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                        +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S"),
                                        "day":"$traffic_info.date"},"avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
            
    else:
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"path":"$_id", "from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S")},
                                "avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        with open("C:/Users/dasan/Desktop/query_new/{}.json".format(filename),'a+') as json_file:
            df.to_json(json_file,orient='table')

In [206]:
#FUNZIONE: ritard_medio_per_hour

hours=["00:00:00","01:00:00","02:00:00","03:00:00","04:00:00","05:00:00","06:00:00","07:00:00",
      "08:00:00","09:00:00","10:00:00","11:00:00","12:00:00","13:00:00","14:00:00","15:00:00",
      "16:00:00","17:00:00","18:00:00","19:00:00","20:00:00","21:00:00","22:00:00","23:00:00",
      "23:59:59"]

k=0
for i in range(len(hours)-1):
    j=i+1
    ritardo_medio_per_hour("prova_def_{}".format(k),"A9 (Como-Lainate)",hours[i],hours[j],
                          month=False)
    k+=1

In [207]:
#da usare per la funzione ritardo_medio_per_hour quando month=False

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_giornaliero_per_tratta.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"path":"{}".format(temp_1[j]["_id"]["path"]),
                          "from-to":"{}".format(temp_1[j]["_id"]["from-to"]),
                          "day":"{}".format(temp_1[j]["_id"]["day"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })

    json.dump(my_data,file)
    file.write("\n")
        #file_1.close()
    #file.close()

for i in range(0,24):
    os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [141]:
#da usare per la funzione ritardo_medio_per_hour quando month=True

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_mensile_per_tratta.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"path":"{}".format(temp_1[j]["_id"]["path"]),
                          "from-to":"{}".format(temp_1[j]["_id"]["from-to"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })

    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(0,24):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [9]:
#FUNZIONE: ritardo_medio_per_day

highways=["A1 (Milano-Napoli)","A10 (Genova-Ventimiglia)","A11 (Firenze-Pisa)","A12 (Genova-Roma)",
         "A12RC (Tarquinia-Maccarese)","A13 (Bologna-Padova)","A14 (Bologna-Taranto)",
          "A15 (Parma-La Spezia)","A16 (Casoria-Canosa di Puglia)","A18 (Messina-Catania)",
          "A19 (Palermo-Catania)","A2 (Salerno-Reggio Calabria)",
          "A20 (Messina-Buonfornello)","A21 (Torino-Brescia)","A22 (Modena-Brennero)",
          "A23 (Palmanova-Tarvisio)","A24 (Roma-Teramo)", "A25 (Torano-Villanova)",
          "A26 (Genova-Sempione)","A27 (Mestre-Pian di Vedoia)","A28 (Portogruaro-Conegliano)",
          "A29 (Palermo-Mazara del Vallo)","A30 (Caserta-Fisciano)","A31 (Rovigo-Piovene Rocchette)",
          "A33 (Asti-Cuneo)","A4 (Torino-Trieste)","A5 (Torino-Traforo Monte Bianco)",
         "A6 (Torino-Savona)","A7 (Milano-Genova)","A9 (Como-Lainate)"]

for i in range(len(highways)):
    ritardo_medio_per_day("prova_def_{}".format(i),highways[i],highway=False)

In [8]:
#da usare per la funzione ritardo_medio_per_day quando highway=True

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_giornaliero_autostrada.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(len(highways)):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":"{}".format(temp_1[j]["_id"]),
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"{}".format(highways[i])
                })

    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(len(highways)):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [10]:
#da usare per la funzione ritardo_medio_per_day quando highway=False

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_giornaliero_autostrada_per_tratta.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(len(highways)):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"path":"{}".format(temp_1[j]["_id"]["path"]),
                           "day":"{}".format(temp_1[j]["_id"]["day"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"{}".format(highways[i])
                })

    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(len(highways)):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [13]:
#FUNZIONE: ritardo_medio_weekday
highways=["A1 (Milano-Napoli)","A10 (Genova-Ventimiglia)","A11 (Firenze-Pisa)","A12 (Genova-Roma)",
         "A12RC (Tarquinia-Maccarese)","A13 (Bologna-Padova)","A14 (Bologna-Taranto)",
          "A15 (Parma-La Spezia)","A16 (Casoria-Canosa di Puglia)","A18 (Messina-Catania)",
          "A19 (Palermo-Catania)","A2 (Salerno-Reggio Calabria)",
          "A20 (Messina-Buonfornello)","A21 (Torino-Brescia)","A22 (Modena-Brennero)",
          "A23 (Palmanova-Tarvisio)","A24 (Roma-Teramo)", "A25 (Torano-Villanova)",
          "A26 (Genova-Sempione)","A27 (Mestre-Pian di Vedoia)","A28 (Portogruaro-Conegliano)",
          "A29 (Palermo-Mazara del Vallo)","A30 (Caserta-Fisciano)","A31 (Rovigo-Piovene Rocchette)",
          "A33 (Asti-Cuneo)","A4 (Torino-Trieste)","A5 (Torino-Traforo Monte Bianco)",
         "A6 (Torino-Savona)","A7 (Milano-Genova)","A9 (Como-Lainate)"]

for i in range(len(highways)):
    ritardo_medio_weekday("prova_def_{}".format(i),highways[i],highway=False)

In [12]:
#da usare per la funzione ritardo_medio_weekday quando highway=True

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_giorno_settimana_autostrada.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(len(highways)):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":"{}".format(temp_1[j]["_id"]),
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"{}".format(highways[i])
                })

    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(len(highways)):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [14]:
#da usare per la funzione ritardo_medio_weekday quando highway=False

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_giorno_settimana_autostrada_per_tratta.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(len(highways)):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"path":"{}".format(temp_1[j]["_id"]["path"]),
                           "day_of_week":"{}".format(temp_1[j]["_id"]["day_of_week"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"{}".format(highways[i])
                })

    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(len(highways)):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [974]:
def get_coordinates(filename,temp_filename,collection,highway):
    mycol=db[collection]
    query=[{
   "$group": {
        "_id":{"path":"$_id",
              "orig_lat":"$origin_latitude",
              "orig_lon":"$origin_longitude",
              "dest_lat":"$destination_latitude",
              "dest_lon":"$destination_longitude"}
   }
}] 
        
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        df.to_json("C:/Users/dasan/Desktop/query/{}.json".format(filename),orient='table')
        
        with open("C:/Users/dasan/Desktop/query/{}.json".format(filename),'r') as file:    
            data=json.load(file)
            temp=data['data']
            with open("C:/Users/dasan/Desktop/query/{}.json".format(temp_filename),'a+') as file_1:
                my_data={}
                my_data['coordinates']=[]
                for i in range(len(temp)):
                    ref=collection.split(" ")[0]
                    if(temp[i]["_id"]["path"]!="{}_coordinates".format(ref)):
                        my_data['coordinates'].append({
                            "path":"{}".format(temp[i]["_id"]["path"]),
                            "orig_lat":"{}".format(temp[i]["_id"]["orig_lat"]),
                            "orig_lon":"{}".format(temp[i]["_id"]["orig_lon"]),
                            "dest_lat":"{}".format(temp[i]["_id"]["dest_lat"]),
                            "dest_lon":"{}".format(temp[i]["_id"]["dest_lon"]),
                            "highway":"{}".format(collection)
                        })
            
                json.dump(my_data,file_1)
                file_1.write("\n")
            
        with open("C:/Users/dasan/Desktop/query/{}.json".format(temp_filename),'r') as my_file:
            data_1=json.load(my_file)
            temp_1=data_1['coordinates']
            df_1=pd.DataFrame(index=range(2*len(temp_1)),
                              columns=["PLACE","LATITUDE","LONGITUDE","PATH","HIGHWAY"])
            j=0
            for i in range(len(temp_1)):
                split_1=temp_1[i]["path"].split("-")
                df_1.iloc[j]["PLACE"]=split_1[0]
                df_1.iloc[j+1]["PLACE"]=split_1[1]
                df_1.iloc[j]["LATITUDE"]=temp_1[i]["orig_lat"]
                df_1.iloc[j]["LONGITUDE"]=temp_1[i]["orig_lon"]
                df_1.iloc[j+1]["LATITUDE"]=temp_1[i]["dest_lat"]
                df_1.iloc[j+1]["LONGITUDE"]=temp_1[i]["dest_lon"]
                df_1.iloc[j]["PATH"]=temp_1[i]["path"]
                df_1.iloc[j+1]["PATH"]=temp_1[i]["path"]
                df_1.iloc[j]["HIGHWAY"]=temp_1[i]["highway"]
                df_1.iloc[j+1]["HIGHWAY"]=temp_1[i]["highway"]
                j=j+2
            
            df_1.to_csv("C:/Users/dasan/Desktop/query/coordinate_autostrade_per_tratta_{}.csv".format(highway),
                        index=False,sep=";")

In [975]:
get_coordinates("prova_def_A1","new_file_A1","A1 (Milano-Napoli)","A1")
get_coordinates("prova_def_A10","new_file_A10","A10 (Genova-Ventimiglia)","A10")
get_coordinates("prova_def_A11","new_file_A11","A11 (Firenze-Pisa)","A11")
get_coordinates("prova_def_A12","new_file_A12","A12 (Genova-Roma)","A12")
get_coordinates("prova_def_A12RC","new_file_A12RC","A12RC (Tarquinia-Maccarese)","A12RC")
get_coordinates("prova_def_A13","new_file_A13","A13 (Bologna-Padova)","A13")
get_coordinates("prova_def_A14","new_file_A14","A14 (Bologna-Taranto)","A14")
get_coordinates("prova_def_A15","new_file_A15","A15 (Parma-La Spezia)","A15")
get_coordinates("prova_def_A16","new_file_A16","A16 (Casoria-Canosa di Puglia)","A16")
get_coordinates("prova_def_A18","new_file_A18","A18 (Messina-Catania)","A18")
get_coordinates("prova_def_A19","new_file_A19","A19 (Palermo-Catania)","A19")
get_coordinates("prova_def_A2","new_file_A2","A2 (Salerno-Reggio Calabria)","A2")
get_coordinates("prova_def_A20","new_file_A20","A20 (Messina-Buonfornello)","A20")
get_coordinates("prova_def_A21","new_file_A21","A21 (Torino-Brescia)","A21")
get_coordinates("prova_def_A22","new_file_A22","A22 (Modena-Brennero)","A22")
get_coordinates("prova_def_A23","new_file_A23","A23 (Palmanova-Tarvisio)","A23")
get_coordinates("prova_def_A24","new_file_A24","A24 (Roma-Teramo)","A24")
get_coordinates("prova_def_A25","new_file_A25","A25 (Torano-Villanova)","A25")
get_coordinates("prova_def_A26","new_file_A26","A26 (Genova-Sempione)","A26")
get_coordinates("prova_def_A27","new_file_A27","A27 (Mestre-Pian di Vedoia)","A27")
get_coordinates("prova_def_A28","new_file_A28","A28 (Portogruaro-Conegliano)","A28")
get_coordinates("prova_def_A29","new_file_A29","A29 (Palermo-Mazara del Vallo)","A29")
get_coordinates("prova_def_A30","new_file_A30","A30 (Caserta-Fisciano)","A30")
get_coordinates("prova_def_A31","new_file_A31","A31 (Rovigo-Piovene Rocchette)","A31")
get_coordinates("prova_def_A33","new_file_A33","A33 (Asti-Cuneo)","A33")
get_coordinates("prova_def_A4","new_file_A4","A4 (Torino-Trieste)","A4")
get_coordinates("prova_def_A5","new_file_A5","A5 (Torino-Traforo Monte Bianco)","A5")
get_coordinates("prova_def_A6","new_file_A6","A6 (Torino-Savona)","A6")
get_coordinates("prova_def_A7","new_file_A7","A7 (Milano-Genova)","A7")
get_coordinates("prova_def_A9","new_file_A9","A9 (Como-Lainate)","A9")

In [3]:
def ritardo_medio_per_hour_weekday(filename,collection,time_start,time_end):
    mycol=db[collection]
    time_start=datetime.datetime.strptime(time_start,"%H:%M:%S")
    time_end=datetime.datetime.strptime(time_end,"%H:%M:%S")
    query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"path":"$_id", "from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                        +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S"),
                                        "day_of_week":"$traffic_info.day_of_week"},"avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        with open("C:/Users/dasan/Desktop/query_new/{}.json".format(filename),'a+') as json_file:
            df.to_json(json_file,orient='table')

In [68]:
#FUNZIONE: ritardo_medio_per_hour_weekday

hours=["00:00:00","01:00:00","02:00:00","03:00:00","04:00:00","05:00:00","06:00:00","07:00:00",
      "08:00:00","09:00:00","10:00:00","11:00:00","12:00:00","13:00:00","14:00:00","15:00:00",
      "16:00:00","17:00:00","18:00:00","19:00:00","20:00:00","21:00:00","22:00:00","23:00:00",
      "23:59:59"]
k=0
for i in range(len(hours)-1):
    j=i+1
    ritardo_medio_per_hour_weekday("prova_def_{}".format(k),"A9 (Como-Lainate)",hours[i],hours[j])
    k+=1

In [69]:
with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_giorno_settimana_per_tratta.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"path":"{}".format(temp_1[j]["_id"]["path"]),
                          "from-to":"{}".format(temp_1[j]["_id"]["from-to"]),
                          "day_of_week":"{}".format(temp_1[j]["_id"]["day_of_week"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })
    
    
    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(0,24):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [15]:
def ritardo_medio_per_hour_highway(filename,collection,time_start,time_end,month=True,
                                   day_of_week=False):
    mycol=db[collection]
    time_start=datetime.datetime.strptime(time_start,"%H:%M:%S")
    time_end=datetime.datetime.strptime(time_end,"%H:%M:%S")
    
    if(month==False):
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                        +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S"),
                                        "day":"$traffic_info.date"},
                                "avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    elif(day_of_week==True):
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                        +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S"),
                                        "day_of_week":"$traffic_info.day_of_week"},
                                "avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
            
    else:
        query= [{"$unwind":"$traffic_info"},
                {"$match": {
                    "$and": [
                    { "traffic_info.time":{"$gte":time_start}},
                    { "traffic_info.time":{"$lte":time_end}}
                   ]
                }},
                    {"$group": { "_id":{"from-to":datetime.datetime.strftime(time_start,"%H:%M:%S")
                                +"-"+datetime.datetime.strftime(time_end,"%H:%M:%S")},
                                "avgDelay": {"$avg":"$traffic_info.delay_minutes"}}}]
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        with open("C:/Users/dasan/Desktop/query_new/{}.json".format(filename),'a+') as json_file:
            df.to_json(json_file,orient='table')

In [204]:
#FUNZIONE: ritard_medio_per_hour_highway

hours=["00:00:00","01:00:00","02:00:00","03:00:00","04:00:00","05:00:00","06:00:00","07:00:00",
      "08:00:00","09:00:00","10:00:00","11:00:00","12:00:00","13:00:00","14:00:00","15:00:00",
      "16:00:00","17:00:00","18:00:00","19:00:00","20:00:00","21:00:00","22:00:00","23:00:00",
      "23:59:59"]

k=0
for i in range(len(hours)-1):
    j=i+1
    ritardo_medio_per_hour_highway("prova_def_{}".format(k),"A9 (Como-Lainate)",hours[i],hours[j],
                                  month=False)
    k+=1

In [205]:
# da usare per ritardo_medio_per_hour_highway quando month=False and day_of_week=False

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_giornaliero_per_autostrada.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{
                        "from-to":"{}".format(temp_1[j]["_id"]["from-to"]),
                        "day":"{}".format(temp_1[j]["_id"]["day"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })
    
    
    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(0,24):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [143]:
# da usare per ritardo_medio_per_hour_highway quando month=True and day_of_week=True

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_giorno_settimana_per_autostrada.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{
                        "from-to":"{}".format(temp_1[j]["_id"]["from-to"]),
                        "day_of_week":"{}".format(temp_1[j]["_id"]["day_of_week"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })
    
    
    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(0,24):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [77]:
# da usare per ritardo_medio_per_hour_highway quando month=True and day_of_week=False

with open("C:/Users/dasan/Desktop/query_new/ritardo_medio_orario_mensile_per_autostrada.json",'a+') as file:
    my_data={}
    my_data['places']=[]
    for i in range(0,24):
        with open("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i),'r') as file_1:
            data_1=json.load(file_1)
            temp_1=data_1['data']
            for j in range(len(temp_1)):
                my_data['places'].append({
                    "index":"{}".format(temp_1[j]["index"]),
                    "_id":{"from-to":"{}".format(temp_1[j]["_id"]["from-to"])},
                    "avgDelay":"{}".format(temp_1[j]["avgDelay"]),
                    "highway":"A9 (Como-Lainate)"
                })
    
    
    json.dump(my_data,file)
    file.write("\n")
            #file_1.close()
        #file.close()


for i in range(0,24):
        os.remove("C:/Users/dasan/Desktop/query_new/prova_def_{}.json".format(i))

In [417]:
def find_minimum_base_time(subfolder):
    
    min_list=[]
    
    directory="C:/Users/dasan/Desktop/DM_project_final_Lagano/{}".format(subfolder)
    for filename in sorted(os.listdir(directory)):
        if(filename.endswith(".csv")):
            df=pd.read_csv(directory+"/"+filename)
            min_base_time_in_file=min(df["BASE TIME (minutes)"])
            min_list.append(min_base_time_in_file)
            
    return min(min_list)

In [418]:
def get_id(collection):
    mycol=db[collection]
    query=[{
   "$group": {
        "_id":"$_id"
   }
}]
    split=collection.split(" ")
    id_list=[]
    
    for i in range(len(list(mycol.aggregate(query)))):
        if(list(mycol.aggregate(query))[i].get("_id") !="{}_coordinates".format(split[0])):
            id_list.append(list(mycol.aggregate(query))[i].get("_id"))
    
    return id_list

In [419]:
def update_db(collection,_id,subfolder):
    mycol=db[collection]
    mycol.update_one(
        {"_id":"{}".format(_id)},
        {"$set":{"base_time":find_minimum_base_time(subfolder)}}
    )

In [456]:
current_collection="A29 (Palermo-Mazara del Vallo)"
query=get_id(current_collection)
print(query)

['Palermo - Alcamo Ovest', 'Alcamo Ovest - Palermo', 'Mazara del Vallo - Alcamo Ovest', 'Alcamo Ovest - Mazara del Vallo']


In [457]:
subfolders=["A29_Palermo_AlcamoOvest","A29_AlcamoOvest_Palermo","A29_MazaraDelVallo_AlcamoOvest",
           "A29_AlcamoOvest_MazaraDelVallo"]

print(len(query))
print(len(subfolders))

4
4


In [458]:
for i in range(len(subfolders)):
    update_db(current_collection,query[i],subfolders[i])

In [464]:
def get_base_time(filename,temp_filename,collection,highway):
    mycol=db[collection]
    query=[{
   "$group": {
        "_id":{"path":"$_id",
              "base_time":"$base_time"}
   }
}] 
        
    
    if not (list(mycol.aggregate(query))):
        print("Data not available")
    else:
        df=pd.DataFrame(list(mycol.aggregate(query)))
        df.to_json("C:/Users/dasan/Desktop/query_2/{}.json".format(filename),orient='table')
        
        with open("C:/Users/dasan/Desktop/query_2/{}.json".format(filename),'r') as file:    
            data=json.load(file)
            temp=data['data']
            with open("C:/Users/dasan/Desktop/query_2/{}.json".format(temp_filename),'a+') as file_1:
                my_data={}
                my_data['coordinates']=[]
                for i in range(len(temp)):
                    ref=collection.split(" ")[0]
                    if(temp[i]["_id"]["path"]!="{}_coordinates".format(ref)):
                        my_data['coordinates'].append({
                            "path":"{}".format(temp[i]["_id"]["path"]),
                            "base_time":"{}".format(temp[i]["_id"]["base_time"]),
                            "highway":"{}".format(collection)
                        })
            
                json.dump(my_data,file_1)
                file_1.write("\n")
            
        with open("C:/Users/dasan/Desktop/query_2/{}.json".format(temp_filename),'r') as my_file:
            data_1=json.load(my_file)
            temp_1=data_1['coordinates']
            df_1=pd.DataFrame(index=range(2*len(temp_1)),
                              columns=["PATH","BASE TIME","HIGHWAY"])
            
            for i in range(len(temp_1)):
                df_1.iloc[i]["PATH"]=temp_1[i]["path"]
                df_1.iloc[i]["BASE TIME"]=temp_1[i]["base_time"]
                df_1.iloc[i]["HIGHWAY"]=temp_1[i]["highway"]
            
            df_1.to_csv("C:/Users/dasan/Desktop/query_2/base_time_autostrade_per_tratta_{}.csv".format(highway),
                        index=False,sep=";")

In [465]:
get_base_time("prova_def_A1","new_file_A1","A1 (Milano-Napoli)","A1")
get_base_time("prova_def_A10","new_file_A10","A10 (Genova-Ventimiglia)","A10")
get_base_time("prova_def_A11","new_file_A11","A11 (Firenze-Pisa)","A11")
get_base_time("prova_def_A12","new_file_A12","A12 (Genova-Roma)","A12")
get_base_time("prova_def_A12RC","new_file_A12RC","A12RC (Tarquinia-Maccarese)","A12RC")
get_base_time("prova_def_A13","new_file_A13","A13 (Bologna-Padova)","A13")
get_base_time("prova_def_A14","new_file_A14","A14 (Bologna-Taranto)","A14")
get_base_time("prova_def_A15","new_file_A15","A15 (Parma-La Spezia)","A15")
get_base_time("prova_def_A16","new_file_A16","A16 (Casoria-Canosa di Puglia)","A16")
get_base_time("prova_def_A18","new_file_A18","A18 (Messina-Catania)","A18")
get_base_time("prova_def_A19","new_file_A19","A19 (Palermo-Catania)","A19")
get_base_time("prova_def_A2","new_file_A2","A2 (Salerno-Reggio Calabria)","A2")
get_base_time("prova_def_A20","new_file_A20","A20 (Messina-Buonfornello)","A20")
get_base_time("prova_def_A21","new_file_A21","A21 (Torino-Brescia)","A21")
get_base_time("prova_def_A22","new_file_A22","A22 (Modena-Brennero)","A22")
get_base_time("prova_def_A23","new_file_A23","A23 (Palmanova-Tarvisio)","A23")
get_base_time("prova_def_A24","new_file_A24","A24 (Roma-Teramo)","A24")
get_base_time("prova_def_A25","new_file_A25","A25 (Torano-Villanova)","A25")
get_base_time("prova_def_A26","new_file_A26","A26 (Genova-Sempione)","A26")
get_base_time("prova_def_A27","new_file_A27","A27 (Mestre-Pian di Vedoia)","A27")
get_base_time("prova_def_A28","new_file_A28","A28 (Portogruaro-Conegliano)","A28")
get_base_time("prova_def_A29","new_file_A29","A29 (Palermo-Mazara del Vallo)","A29")
get_base_time("prova_def_A30","new_file_A30","A30 (Caserta-Fisciano)","A30")
get_base_time("prova_def_A31","new_file_A31","A31 (Rovigo-Piovene Rocchette)","A31")
get_base_time("prova_def_A33","new_file_A33","A33 (Asti-Cuneo)","A33")
get_base_time("prova_def_A4","new_file_A4","A4 (Torino-Trieste)","A4")
get_base_time("prova_def_A5","new_file_A5","A5 (Torino-Traforo Monte Bianco)","A5")
get_base_time("prova_def_A6","new_file_A6","A6 (Torino-Savona)","A6")
get_base_time("prova_def_A7","new_file_A7","A7 (Milano-Genova)","A7")
get_base_time("prova_def_A9","new_file_A9","A9 (Como-Lainate)","A9")

In [470]:
df_1=pd.read_csv("C:/Users/dasan/Desktop/query/coordinate_autostrade_per_tratta.csv",sep=";")
df_2=pd.read_csv("C:/Users/dasan/Desktop/query_2/base_time_autostrade_per_tratta.csv",sep=";")

df_3=pd.DataFrame(index=range(len(df_1)),
                              columns=["PLACE","LATITUDE","LONGITUDE","BASE TIME","PATH","HIGHWAY"])

#print(df_3)

j=0
for i in range(len(df_3)):
    df_3.iloc[i]["PLACE"]=df_1.iloc[i]["PLACE"]
    df_3.iloc[i]["LATITUDE"]=df_1.iloc[i]["LATITUDE"]
    df_3.iloc[i]["LONGITUDE"]=df_1.iloc[i]["LONGITUDE"]
    df_3.iloc[i]["PATH"]=df_1.iloc[i]["PATH"]
    df_3.iloc[i]["HIGHWAY"]=df_1.iloc[i]["HIGHWAY"]
    
    path=df_2.iloc[j]["PATH"]
    for k in range(len(df_1)):
        if(df_1.iloc[k]["PATH"]==path):
            df_3.iloc[i]["BASE TIME"]=df_2.iloc[j]["BASE TIME"]
    
    if(i%2!=0):
        j+=1
            
df_3.to_csv("C:/Users/dasan/Desktop/info_autostrade.csv",index=False,sep=";")
    
    

In [471]:
df=pd.read_csv("C:/Users/dasan/Desktop/info_autostrade.csv",sep=";")

In [472]:
df

,PLACE,LATITUDE,LONGITUDE,BASE TIME,PATH,HIGHWAY
0,Roncobilaccio,44.117249,11.221295,39.0,Roncobilaccio - Sasso Marconi Nord,A1 (Milano-Napoli)
1,Sasso Marconi Nord,44.450105,11.280348,39.0,Roncobilaccio - Sasso Marconi Nord,A1 (Milano-Napoli)
2,Sasso Marconi Nord,44.450105,11.280348,39.0,Sasso Marconi Nord - Roncobilaccio,A1 (Milano-Napoli)
3,Roncobilaccio,44.117249,11.221295,39.0,Sasso Marconi Nord - Roncobilaccio,A1 (Milano-Napoli)
4,Roncobilaccio,44.117249,11.221295,38.0,Roncobilaccio - Firenze Sud,A1 (Milano-Napoli)
5,Firenze Sud,43.739283,11.272574,38.0,Roncobilaccio - Firenze Sud,A1 (Milano-Napoli)
6,Pontecorvo,41.461618,13.666709,66.0,Pontecorvo - Caianello,A1 (Milano-Napoli)
7,Caianello,41.300234,14.088442,66.0,Pontecorvo - Caianello,A1 (Milano-Napoli)
8,Pontecorvo,41.461618,13.666709,58.0,Pontecorvo - Anagni,A1 (Milano-Napoli)
9,Anagni,41.744720,13.152694,58.0,Pontecorvo - Anagni,A1 (Milano-Napoli)
